In [1]:
import sys
sys.path.append('/fsx/matzeni/duck')

In [4]:
import torch
import h5py
import json
from pathlib import Path
import pickle
from tqdm import tqdm
import logging
from transformers import AutoTokenizer
from typing import Any, Dict, List, Optional, Tuple
import copy
from einops import rearrange, repeat
import numpy as np
from duck.box_tensors import BoxTensor
from duck.task.duck_entity_disambiguation import Duck
from hydra import compose, initialize
import hydra
from duck.common.utils import seed_prg
import collections
import logging
import math
from duck.common.utils import load_json, load_jsonl
from torch import nn
from duck.modules import BoxEmbedding
from duck.box_tensors.volume import Volume
import random
from duck.box_tensors.initializers.gaussian_margin import gaussian_margin_boxes
import matplotlib.pyplot as plt
logger = logging.getLogger()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# plt.style.use('seaborn')
plt.rcParams['axes.labelsize'] = 'large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize'] = 'medium'
plt.rcParams['ytick.labelsize'] = 'medium'
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.color'] = "#dbdbdb"

In [ ]:
size = 1024
left, right = gaussian_margin_boxes(
    dimensions=1024,
    num_boxes=5,
    minimum = -8,
    maximum = +8,
    stddev=1,
    cut=False
)
vol = Volume(intersection_temperature=0.0001, volume_temperature=0.1)
box = BoxTensor.sigmoid_constructor(left, right)
print(left, right)
print(box)
universe = BoxTensor((
    torch.zeros(size), torch.ones(size)
))
print(vol(universe).exp())
print("Prob:", (vol(box) - vol(universe)).exp())


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(right.view(-1).cpu().numpy(), density=True, color="#4fbfff", bins=100)
plt.hist(left.view(-1).cpu().numpy(), density=True, color="#8cff92", bins=100)
plt.xlabel("Parameter values")
x_sigmoid = torch.arange(-10, 10, 0.01)
y_sigmoid = torch.sigmoid(x_sigmoid)
# plt.plot(x_sigmoid, y_sigmoid)
plt.show()

In [ ]:
torch.cat([left.view(-1), right.view(-1)]).std()

In [ ]:
math.sqrt(1.0 / 1120)

In [ ]:
math.sqrt(2.0 / float(1120 + 1024))

In [ ]:
torch.sigmoid(torch.tensor(-100))

In [ ]:
margin = 1e-7
box = BoxTensor((
    torch.zeros(1024) + margin,
    torch.ones(1024) - margin
))
(vol(box) - vol(universe)).exp()